In [1]:
import pandas as pd

predicted_emotions = pd.read_csv('predicted_emotions.csv')
predicted_emotions_painting_and_utterance = pd.read_csv('predicted_emotions_painting_and_utterance.csv')
predicted_emotions_only_on_utterance = pd.read_csv('predicted_emotions_only_on_utterance.csv')
original_dataset = pd.read_csv('1k_batch.csv')

merged_results = original_dataset[['painting', 'emotion']].merge(
    predicted_emotions[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_predicted')
).merge(
    predicted_emotions_painting_and_utterance[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_painting_and_utterance')
).merge(
    predicted_emotions_only_on_utterance[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_only_on_utterance')
)

def calculate_accuracy(row):
    return {
        'original': row['emotion'],
        'only_image': row['predicted_emotion'],
        'image_and_utterance': row['predicted_emotion_painting_and_utterance'],
        'only_utterance': row['predicted_emotion_only_on_utterance'],
        'correct_only_image': row['emotion'] in row['predicted_emotion'],
        'correct_image_and_utterance': row['emotion'] in row['predicted_emotion_painting_and_utterance'],
        'correct_only_utterance': row['emotion'] in row['predicted_emotion_only_on_utterance']
    }

comparison_results = merged_results.apply(calculate_accuracy, axis=1, result_type='expand')

accuracy_only_image = comparison_results['correct_only_image'].mean()
accuracy_image_and_utterance = comparison_results['correct_image_and_utterance'].mean()
accuracy_only_utterance = comparison_results['correct_only_utterance'].mean()

print(f"Accuracy (Only Image): {accuracy_only_image:.2%}")
print(f"Accuracy (Image and Utterance): {accuracy_image_and_utterance:.2%}")
print(f"Accuracy (Only Utterance): {accuracy_only_utterance:.2%}")

comparison_results.to_csv('comparison_results.csv', index=False)


Accuracy (Only Image): 36.90%
Accuracy (Image and Utterance): 10.00%
Accuracy (Only Utterance): 6.60%


In [2]:
import pandas as pd

predicted_emotions = pd.read_csv('predicted_emotions.csv')
predicted_emotions_painting_and_utterance = pd.read_csv('predicted_emotions_painting_and_utterance.csv')
predicted_emotions_only_on_utterance = pd.read_csv('predicted_emotions_only_on_utterance.csv')
original_dataset = pd.read_csv('1k_batch.csv')

merged_results = original_dataset[['painting', 'emotion', 'utterance']].merge(
    predicted_emotions[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_predicted')
).merge(
    predicted_emotions_painting_and_utterance[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_painting_and_utterance')
).merge(
    predicted_emotions_only_on_utterance[['painting', 'predicted_emotion', 'similarity']], on='painting', suffixes=('', '_only_on_utterance')
)

merged_results['correct_only_image'] = merged_results['emotion'] == merged_results['predicted_emotion']
merged_results['correct_image_and_utterance'] = merged_results['emotion'] == merged_results['predicted_emotion_painting_and_utterance']
merged_results['correct_only_utterance'] = merged_results['emotion'] == merged_results['predicted_emotion_only_on_utterance']

merged_results.to_csv('merged_comparison_results.csv', index=False)

incorrect_cases_only_image = merged_results[~merged_results['correct_only_image']]
incorrect_cases_image_and_utterance = merged_results[~merged_results['correct_image_and_utterance']]
incorrect_cases_only_utterance = merged_results[~merged_results['correct_only_utterance']]

print("Incorrect cases (Only Image):")
print(incorrect_cases_only_image[['painting', 'emotion', 'utterance', 'predicted_emotion']].head())

print("\nIncorrect cases (Image and Utterance):")
print(incorrect_cases_image_and_utterance[['painting', 'emotion', 'utterance', 'predicted_emotion_painting_and_utterance']].head())

print("\nIncorrect cases (Only Utterance):")
print(incorrect_cases_only_utterance[['painting', 'emotion', 'utterance', 'predicted_emotion_only_on_utterance']].head())

emotion_distribution = original_dataset['emotion'].value_counts()
print("\nEmotion distribution in the dataset:")
print(emotion_distribution)


Incorrect cases (Only Image):
                                            painting    emotion  \
0              agostino-carracci_venus-and-mars-1600  amusement   
1                    diego-velazquez_self-portrait-1  amusement   
2                        felicien-rops_tienne-soubre  amusement   
3      koloman-moser_animal-motif-for-a-picture-book  amusement   
4  martiros-saryan_woman-with-mask-s-i-dymshits-1913  amusement   

                                           utterance  \
0  All of the figures appear to have very importa...   
1  The mans collar looks like a college graduatio...   
2  The head is ridiculously large and the legs ar...   
3  The little green shape reminds me of a bird an...   
4  The facial expression indicates that woman is ...   

                                predicted_emotion  
0  This artwork conveys a feeling of contentment.  
1      This artwork conveys a feeling of disgust.  
2      This artwork conveys a feeling of disgust.  
3  This artwork convey

In [8]:
import pandas as pd
from sklearn.metrics import classification_report

predicted_emotions = pd.read_csv('predicted_emotions.csv')
predicted_emotions_painting_and_utterance = pd.read_csv('predicted_emotions_painting_and_utterance.csv')
predicted_emotions_only_on_utterance = pd.read_csv('predicted_emotions_only_on_utterance.csv')
original_dataset = pd.read_csv('1k_batch.csv')

merged_results = original_dataset[['painting', 'emotion']].merge(
    predicted_emotions[['painting', 'predicted_emotion']], on='painting', suffixes=('', '_predicted')
).merge(
    predicted_emotions_painting_and_utterance[['painting', 'predicted_emotion']], on='painting', suffixes=('', '_painting_and_utterance')
).merge(
    predicted_emotions_only_on_utterance[['painting', 'predicted_emotion']], on='painting', suffixes=('', '_only_on_utterance')
)

def extract_emotion_label(full_label):
    emotion_map = {
        "This artwork conveys a feeling of amusement.": "amusement",
        "This artwork conveys a feeling of anger.": "anger",
        "This artwork conveys a feeling of awe.": "awe",
        "This artwork conveys a feeling of contentment.": "contentment",
        "This artwork conveys a feeling of disgust.": "disgust",
        "This artwork conveys a feeling of excitement.": "excitement",
        "This artwork conveys a feeling of fear.": "fear",
        "This artwork conveys a feeling of sadness.": "sadness"
    }
    return emotion_map.get(full_label, "unknown")

merged_results['predicted_emotion'] = merged_results['predicted_emotion'].apply(extract_emotion_label)
merged_results['predicted_emotion_painting_and_utterance'] = merged_results['predicted_emotion_painting_and_utterance'].apply(extract_emotion_label)
merged_results['predicted_emotion_only_on_utterance'] = merged_results['predicted_emotion_only_on_utterance'].apply(extract_emotion_label)

y_true = merged_results['emotion']

def generate_classification_report(y_true, y_pred, title):
    report = classification_report(y_true, y_pred, output_dict=True)
    report_df = pd.DataFrame(report).transpose()

    print(f"Classification Report: {title}")
    print(report_df)
    report_df.to_csv(f'{title}_classification_report.csv', index=True)

y_pred_only_image = merged_results['predicted_emotion']
generate_classification_report(y_true, y_pred_only_image, 'only_image')

y_pred_image_and_utterance = merged_results['predicted_emotion_painting_and_utterance']
generate_classification_report(y_true, y_pred_image_and_utterance, 'image_and_utterance')

y_pred_only_utterance = merged_results['predicted_emotion_only_on_utterance']
generate_classification_report(y_true, y_pred_only_utterance, 'only_utterance')


Classification Report: only_image
              precision    recall  f1-score   support
amusement      0.354167  0.128788  0.188889   132.000
anger          0.025000  0.142857  0.042553     7.000
awe            0.290640  0.357576  0.320652   165.000
contentment    0.530786  0.668449  0.591716   374.000
disgust        0.066667  0.205128  0.100629    39.000
excitement     0.122807  0.112903  0.117647    62.000
fear           0.500000  0.203704  0.289474   108.000
sadness        0.416667  0.044248  0.080000   113.000
unknown        0.000000  0.000000  0.000000     0.000
accuracy       0.369000  0.369000  0.369000     0.369
macro avg      0.256304  0.207073  0.192396  1000.000
weighted avg   0.404692  0.369000  0.350962  1000.000
Classification Report: image_and_utterance
              precision    recall  f1-score  support
amusement      0.181818  0.136364  0.155844    132.0
anger          0.000000  0.000000  0.000000      7.0
awe            0.049180  0.018182  0.026549    165.0
contentme

e:\大三下课程\python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\大三下课程\python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\大三下课程\python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
